In [16]:
#Kind of brute force method. Taking all data from cooldown and using that to generate
#a txt file that Nick's code can read. 
#Avg temp function edited by LTL32 May/June 2023

%matplotlib inline
from so3g.hk import load_range
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv 
# matplotlib.use('Agg')

In [2]:
def find_index_jump(timeseries):
    """Function to look for places in the data taking analysis where the 
       timeseries of the resistances jumps more than a second or two.
       This corresponds to the places where measurements stopped or started."""

    timestamps = []

    for i in range(1,len(timeseries)):
        if(timeseries[i]-timeseries[i-1] > 2):
            timestamps.append(i)

    return timestamps

def timestamp_of_jumps(timeseries, jumps):
    """Function to find the start and stop times of each resistance
	   measurement on a channel. Output as a list of tuples, where
	   each tuple contains the start and stop timestamp.
	"""
    times = []

    # Handling the first timestamp
    times.append((timeseries[0],timeseries[jumps[0]-1]))

    for i in range(len(jumps)-1):
        start_and_stop = (timeseries[jumps[i]],timeseries[jumps[i+1]-1])
        times.append(start_and_stop)

    # Handling the first timestamp
    times.append((timeseries[jumps[-1]],timeseries[-1]))

    return times

def calc_avg_res(resistances,jump_index):
    """Function to calculate the average resistance in a series of
	   resistance data from a Grafana feed given a list of the indices
	   where the resistance measurements start.
	"""
    avg_resistances = []
    std_resistances = []

    # Handling first point
    avg_resistances.append(np.mean(resistances[0:jump_index[0]]))
    std_resistances.append(np.std(resistances[0:jump_index[0]]))

    for i in range(len(jump_index)-1):
        avg_resistances.append(np.mean(resistances[jump_index[i]:jump_index[i+1]]))
        std_resistances.append(np.std(resistances[jump_index[i]:jump_index[i+1]]))

    # Handling last point - definitely should be a better way to do this...
    avg_resistances.append(np.mean(resistances[jump_index[-1]:]))
    std_resistances.append(np.std(resistances[jump_index[-1]:]))    

    return avg_resistances, std_resistances	

def calc_avg_temp(temp_data,jump_index):
    """Function to calculate the average temperature in a time range
	   given by a tuple or nested array of start and stop timestamps."""
    avg_temps = []
    std_temps = []

    # Handling first point
    avg_temps.append(np.mean(temp_data[0:jump_index[0]]))
    std_temps.append(np.std(temp_data[0:jump_index[0]]))

    for i in range(len(jump_index)-1):
        avg_temps.append(np.mean(temp_data[jump_index[i]:jump_index[i+1]]))
        std_temps.append(np.std(temp_data[jump_index[i]:jump_index[i+1]]))

    # Handling last point - definitely should be a better way to do this...
    avg_temps.append(np.mean(temp_data[jump_index[-1]:]))
    std_temps.append(np.std(temp_data[jump_index[-1]:]))    

    return avg_temps, std_temps	
#     avg_temps = []
#     std_temps = []

#     times_T = temp_data[0]
#     temps_T = temp_data[1]

#     # Step 1: use times from the start and stop of the resistance measurement
#     #         to get the corresponding start and stop indices of the temperature timestamps
#     for time in times:
#         start = time[0]
#         stop = time[1]
        
#         start_index = next(x for x, val in enumerate(temps[0]) if (val > start))
#         stop_index = next(x for x, val in enumerate(temps[0]) if (val > stop))

#         # Step 2: use the temperature timestamps to extract the temperatures and average
#         avg_temps.append(np.mean(temps_T[start_index:stop_index]))
#         std_temps.append(np.std(temps_T[start_index:stop_index]))

#     return avg_temps, std_temps


In [23]:
fields = []
channels = [6,7,8]
for ch in channels:
	fields.append('observatory.LSA2HON.feeds.temperatures.Channel_'+str(0)+str(ch)+'_R')

fields.append('observatory.LSA2HON.feeds.temperatures.Channel_06_T')


start = dt.datetime(2024,10,15,8,0,tzinfo=dt.timezone.utc)

file_name = dt.datetime(2024,10,15,8,0,tzinfo=dt.timezone.utc).strftime("%Y_%m_%d_%H_%M_%S")

# stop = dt.datetime(2023,5,11,1,0,tzinfo=dt.timezone.utc)
stop = dt.datetime(2024,10,16,3,0,tzinfo=dt.timezone.utc)

data = load_range(start,stop,fields=fields,data_dir='./hk/hk/')


avgs_R = [[],[],[]]
timeseries = []
resistance = []
jump_index = []
jump_times = []
avgs_T = [[]]
#index 0 is 6, 1-7, 2-8. 
for i in range(len(channels)):
    # timeseries[i] = data[fields[i]][0]
    timeseries.append(data[fields[i]][0])
    # resistance[i] = data[fields[i]][1]
    resistance.append(data[fields[i]][1])
    # jump_index[i] = find_index_jump(timeseries[i])
    jump_index.append(find_index_jump(timeseries[i]))
    avgs_R[i], stds_R = calc_avg_res(resistance[i],jump_index[i])
    jump_times.append(timestamp_of_jumps(timeseries[i],jump_index[i]))
# print(resistance[0])
# print(jump_index[0])
temps = []
temps.append(data[fields[3]][1])
# print(temps[0])
# print(temps[0][0]) This is time index [1] is temperature data.
avgs_T[0], stds_T = calc_avg_temp(temps[0],jump_index[0])

In [24]:
#for ch6,7,8

fields = ['', 'ctime', 'ch', 'res','temp']     
bychannel=[]

for j in range(len(channels)):
    temp = []
    #Add in temperature for ch6
    if j == 0:
        for i in range(len(jump_times[j])):
            temp.append([i,jump_times[j][i][0],j+6,avgs_R[j][i],avgs_T[0][i]])
        bychannel.append(temp)
    else:
        for i in range(len(jump_times[j])):
            temp.append([i,jump_times[j][i][0],j+6,avgs_R[j][i],0])
        bychannel.append(temp)
    
# name of csv file
for ch in range(len(bychannel)):
    actualChannel = ch+6
    filename = "./csvOutput/SSBDR_cal_data_"+str(file_name)+"_ch%d.csv"%actualChannel
    
# writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(bychannel[ch])

In [4]:
fields = []
channels = [2,3,4]
for ch in channels:
	fields.append('observatory.LSA2HON.feeds.temperatures.Channel_'+str(0)+str(ch)+'_R')

fields.append('observatory.LSA2HON.feeds.temperatures.Channel_02_T')

# start = dt.datetime(2023,5,9,11,0,tzinfo=dt.timezone.utc)

# file_name = dt.datetime(2023,5,9,11,0,tzinfo=dt.timezone.utc).strftime("%Y_%m_%d_%H_%M_%S")



start = dt.datetime(2024,10,6,12,0,tzinfo=dt.timezone.utc)

file_name = dt.datetime(2024,10,6,12,0,tzinfo=dt.timezone.utc).strftime("%Y_%m_%d_%H_%M_%S")

# stop = dt.datetime(2023,5,11,1,0,tzinfo=dt.timezone.utc)
stop = dt.datetime(2024,10,8,23,0,tzinfo=dt.timezone.utc)

data = load_range(start,stop,fields=fields,data_dir='./hk/hk/')


avgs_R = [[],[],[]]
timeseries = []
resistance = []
jump_index = []
jump_times = []
avgs_T = [[]]
#index 0 is 6, 1-7, 2-8. 
for i in range(len(channels)):
    # timeseries[i] = data[fields[i]][0]
    timeseries.append(data[fields[i]][0])
    # resistance[i] = data[fields[i]][1]
    resistance.append(data[fields[i]][1])
    # jump_index[i] = find_index_jump(timeseries[i])
    jump_index.append(find_index_jump(timeseries[i]))
    avgs_R[i], stds_R = calc_avg_res(resistance[i],jump_index[i])
    jump_times.append(timestamp_of_jumps(timeseries[i],jump_index[i]))
# print(resistance[0])
# print(jump_index[0])
temps = []
temps.append(data[fields[3]][1])
# print(temps[0])
# print(temps[0][0]) This is time index [1] is temperature data.
avgs_T[0], stds_T = calc_avg_temp(temps[0],jump_index[0])

In [5]:
#for ch 2,3,4

fields = ['', 'ctime', 'ch', 'res','temp']     
bychannel=[]

for j in range(len(channels)):
    temp = []
    #Add in temperature for ch6
    if j == 0:
        for i in range(len(jump_times[j])):
            temp.append([i,jump_times[j][i][0],j+2,avgs_R[j][i],avgs_T[0][i]])
        bychannel.append(temp)
    else:
        for i in range(len(jump_times[j])):
            temp.append([i,jump_times[j][i][0],j+2,avgs_R[j][i],0])
        bychannel.append(temp)
    
# name of csv file
for ch in range(len(bychannel)):
    actualChannel = ch+2
    filename = "./csvOutput/SSBDR_cal_data_"+str(file_name)+"_ch%d.csv"%actualChannel
    
# writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(bychannel[ch])

In [59]:
# plt.plot(jump_times[0][0:10],avgs_T[0][0:10])
# # plt.plot(timeseries[1],resistance[1])
# plt.title("Temp vs Time")
# plt.xlabel("Time")
# plt.ylabel('Resistance  ($m\Omega$)')
# plt.show()

In [60]:
# data['observatory.LSA2HON.feeds.temperatures.Channel_07_R'][0]
#print(jump_index[2])

In [61]:
# plt.plot(timeseries[0],resistance[0])
# # plt.plot(timeseries[1],resistance[1])
# plt.title("Resistance vs Time")
# plt.xlabel("Time")
# plt.ylabel('Resistance  ($m\Omega$)')
# plt.show()

In [62]:
# plt.plot(jump_times[0][0:10],avgs_R[0][0:10])
# plt.title("Temp vs Time")
# plt.xlabel("Time")
# plt.ylabel('Resistance  ($m\Omega$)')
# plt.show()

In [4]:
fields = []
channels = [1]
for ch in channels:
	fields.append('observatory.LSA2HON.feeds.temperatures.Channel_'+str(0)+str(ch)+'_R')

fields.append('observatory.LSA2HON.feeds.temperatures.Channel_01_T')

# start = dt.datetime(2023,5,9,11,0,tzinfo=dt.timezone.utc)

# file_name = dt.datetime(2023,5,9,11,0,tzinfo=dt.timezone.utc).strftime("%Y_%m_%d_%H_%M_%S")



start = dt.datetime(2024,3,16,0,0,tzinfo=dt.timezone.utc)

file_name = dt.datetime(2024,3,16,0,0,tzinfo=dt.timezone.utc).strftime("%Y_%m_%d_%H_%M_%S")

# stop = dt.datetime(2023,5,11,1,0,tzinfo=dt.timezone.utc)
stop = dt.datetime(2024,4,1,0,0,tzinfo=dt.timezone.utc)

data = load_range(start,stop,fields=fields,data_dir='./hk/hk/')


avgs_R = [[],[],[]]
timeseries = []
resistance = []
jump_index = []
jump_times = []
avgs_T = [[]]
#index 0 is 6, 1-7, 2-8. 
for i in range(len(channels)):
    # timeseries[i] = data[fields[i]][0]
    timeseries.append(data[fields[i]][0])
    # resistance[i] = data[fields[i]][1]
    resistance.append(data[fields[i]][1])
    # jump_index[i] = find_index_jump(timeseries[i])
    jump_index.append(find_index_jump(timeseries[i]))
    avgs_R[i], stds_R = calc_avg_res(resistance[i],jump_index[i])
    jump_times.append(timestamp_of_jumps(timeseries[i],jump_index[i]))
# print(resistance[0])
# print(jump_index[0])
temps = []
temps.append(data[fields[1]][1])
# print(temps[0])
# print(temps[0][0]) This is time index [1] is temperature data.
avgs_T[0], stds_T = calc_avg_temp(temps[0],jump_index[0])
#for ch 2,3,4

fields = ['', 'ctime', 'ch', 'res','temp']     
bychannel=[]

for j in range(len(channels)):
    temp = []
    #Add in temperature for ch6
    if j == 0:
        for i in range(len(jump_times[j])):
            temp.append([i,jump_times[j][i][0],channels[0],avgs_R[j][i],avgs_T[0][i]])
        bychannel.append(temp)
    else:
        for i in range(len(jump_times[j])):
            temp.append([i,jump_times[j][i][0],channels[0],avgs_R[j][i],0])
        bychannel.append(temp)
    
# name of csv file
for ch in range(len(bychannel)):
    actualChannel = channels[0]
    filename = "./csvOutput/SSBDR_cal_data_"+str(file_name)+"_ch%d.csv"%actualChannel
    
# writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(bychannel[ch])